# Simple example illustrating basic aspects of ridge linear regression

In [ ]:
# Uncomment to upgrade packages
#!pip3 install pandas --upgrade --user --quiet
#!pip3 install numpy --upgrade --user --quiet
#!pip3 install scikit-learn --upgrade --user --quiet

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 4)

## 1. Load `murders.txt` data from class

In [ ]:
df = pd.read_csv("murders.txt", sep=" ")
df["x_0"] = 1
print(df)

In [ ]:
# put data into X matrix and y column

X = df[["x_0", "inhabitants", "poverty", "unemployment"]].to_numpy()
y = df[["murders"]].to_numpy()

print(X.shape, y.shape)

## 2. Train/test split for later validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=0
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## 3. Preprocessing: first normalize training set, then apply same transformation to test

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(
    X_train
)  # computes means and stdevs for each column in X_train
X_train_scaled = scaler.transform(
    X_train
)  # substracts mean and divides by stdev (estimated from training)
X_test_scaled = scaler.transform(
    X_test
)  # substracts mean and divides by stdev (estimated from training)

X_train_scaled[:, 0] = 1  # undo transformation for all-1 column
X_test_scaled[:, 0] = 1  # undo transformation for all-1 column

print(X_train_scaled.mean(axis=0))
print(X_test_scaled.mean(axis=0))

__Question__: why are means for X_test_scaled not zero? and for X_train_scaled?

## 4. Train ridge regression model for several $\lambda$ values

As explained in class, for each value of $\lambda$ that we want to try, 
we use _maximum a posteriori_ method to find linear coefficients $\theta_0, \theta_1, ..$:

$$\theta_{ridge} = (X^T X + \lambda I)^{-1} X^T y$$

### train model, using formulas from slides

1. For each $\lambda\in\Lambda$:
    - compute optimal solution $\hat{\mathbf{\theta}}_{\lambda} = (\mathbf{X}^T\mathbf{X} + \lambda I)^{-1}\mathbf{X}^T\mathbf{y}$ 
    - compute "hat" matrix $H_{\lambda} = \mathbf{X}(\mathbf{X}^T\mathbf{X} + \lambda I)^{-1}\mathbf{X}^T$
    - compute loocv directly for each $\lambda$ (no need to use folds etc.):

    $$loocv(\lambda) = \frac{1}{n} \sum_{i=1}^n \left( \frac{y_i - \mathbf{x}_i \mathbf{\theta}_{\lambda}}{1 - h_{ii}}  \right)^2$$
    
2. Return $\lambda$ with minimum *loocv*


In [ ]:
from sklearn.metrics import mean_squared_error

lambdas = np.logspace(start=-4, stop=1.1, num=100, base=10.0)
results = []
X = X_train_scaled.copy()
y = y_train.copy()
n = y.shape[0]
d = X.shape[1]

for l in lambdas:
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX + l * np.identity(n=d))
    coefs = (XtX_inv) @ X.T @ y
    hatmat = X @ XtX_inv @ X.T
    trace_hatmat = np.trace(hatmat)
    y_pred = X @ coefs
    loocv = (
        1
        / n
        * np.sum([((y[i] - y_pred[i]) / (1 - hatmat[i, i])) ** 2 for i in range(n)])
    )
    mse = mean_squared_error(y, y_pred)
    gcv = mse / (1 - trace_hatmat / n) ** 2
    results.append([l, mse, loocv, gcv])

df = pd.DataFrame(results, columns=["lambda", "training_mse", "loocv", "gcv"])
df.sort_values(by="loocv")

In [ ]:
df.plot(kind="line", x="lambda");

### select best lambda according to loocv; re-train on the whole training set to obtain model using best lambda

In [ ]:
best_lambda = df.loc[df["loocv"].idxmin()]["lambda"]

print(f"best lambda value: {best_lambda:.4f}")

# apply formula with "best lambda"
theta_vector = (
    np.linalg.inv(X_train_scaled.T @ X_train_scaled + best_lambda * np.identity(n=d))
    @ X_train_scaled.T
    @ y
)

### as a sanity check, see what RidgeRegressionCV from scikit-learn obtains as best lambda

In [ ]:
from sklearn.linear_model import RidgeCV

print(f"there are {X_train_scaled.shape[0]} training examples.")
results = []
for k in range(2, 6 + 1):
    ridge = RidgeCV(alphas=lambdas, fit_intercept=False, cv=k)  # k-fold cross-val
    clf = ridge.fit(X_train_scaled, y_train)
    results.append([k, clf.alpha_])

## "efficient"  way:
ridge = RidgeCV(alphas=lambdas, fit_intercept=False, cv=None)
clf = ridge.fit(X_train_scaled, y_train)
results.append(["efficient", clf.alpha_])

pd.DataFrame(results, columns=["cross-val method (k)", "best lambda"])

## 4. Test trained linear model to predict `murders` on __unseen data__  (test partition)

In [ ]:
# make predictions on unseen test points (from test partition)
y_pred = X_test_scaled @ theta_vector

# compute squared error for each example
abs_error = np.abs(y_pred - y_test)
sq_error = np.square(y_pred - y_test)

# show them in table
dict_data = {
    "target": y_test.ravel(),
    "pred": y_pred.ravel(),
    "abs_error": abs_error.ravel(),
    "squared_error": sq_error.ravel(),
}
df = pd.DataFrame.from_dict(dict_data)
print(df)

print(f"\nThe TEST mean squared error is: {mean_squared_error(y_pred, y_test)}")

In [ ]:
ax = df.plot.scatter(x="target", y="pred", c="abs_error")
ax.axis("equal");